In [3]:
import json
import regex
import collections
import numpy as np
import pandas as pd

## Extract ontology events

In [4]:
df = pd.read_excel('KAIROS_Annotation_Tagset_Phase_1_V1.0_argsplit.xlsx', sheet_name='events')

In [5]:
events = {}

for row in df.iterrows():
    row_id, row = row
    event_type = row['Type']
    event_subtype = row['Subtype']
    event_subsubtype = row['Sub-subtype']
    event_definition = row['Definition']
    event_template = row['Template']
    event_args = []
    arg_id = 1
    while True:
        if f'arg{arg_id} label' not in row:
            break
        arg_label = row[f'arg{arg_id} label']
        if pd.isna(arg_label):
            break
        arg_label = arg_label.strip()
        arg_constraints_str = row[f'arg{arg_id} type constraints']
        arg_constraints = [s.strip() for s in arg_constraints_str.strip().split(',')]
        event_args.append((arg_label, arg_constraints))
        arg_id += 1
    if event_subtype == 'Unspecified':
        event_name = event_type
    elif event_subsubtype == 'Unspecified':
        event_name = f'{event_type}.{event_subtype}'
    else:
        event_name = f'{event_type}.{event_subtype}.{event_subsubtype}'
    
    assert event_name not in events
    events[event_name] = {
        'definition': event_definition,
        'template': event_template,
        'args': event_args
    }

In [6]:
def to_short_name(event_name):
    event_type, rest = event_name.split('.', 1)
    event_abbr = ''.join([c for c in event_type if c.isupper()])
    return f'{event_abbr}.{rest}'

## Generate Blockly blocks

In [7]:
# Generate JSON events args

events_args = {k: v['args'] for k, v in events.items()}

events_parts_args = collections.defaultdict(list)

for event_name, event_body in events.items():
    event_args = event_body['args']
    event_body_parts = event_body['template'].split('|')
    for i, body_part in enumerate(event_body_parts):
        body_part_args = []
        for arg_id_str in regex.findall('<arg(\d+)>', body_part):
            arg_id = int(arg_id_str) - 1
            body_part_args.append(arg_id)
        events_parts_args[event_name].append(body_part_args)

with open('../kairos_events_args.js', 'w') as fout:
    fout.write(f'var events_args = {json.dumps(events_args)};\n')
    fout.write(f'var events_parts_args = {json.dumps(events_parts_args)};\n')

In [16]:
events_connections = [f'kairos_event_{event_name}' for event_name in events.keys()] + ['kairos_control_parallel']

def event_to_blocks(event_name, event_body):
    event_template = event_body['template']
    body_parts = event_template.split('|')

    # create top block for a given event
    parent_block = {
        'type': f'kairos_event_{event_name}',
        'message0': '%1 %2',
        'args0': [{
            'type': 'field_input',
            'name': 'step_name',
        }],
        'previousStatement': events_connections,
        'nextStatement': events_connections,
        'style': 'loop_blocks',  # TODO: change
        'inputsInline': False,
        'tooltip': event_name + ":\n" + event_body['definition'],
    }
    part_blocks = []
    # replace <argI> i=1.. with arg names
    # split by |, and create separate blocks for each body part
    for i, body_part in enumerate(body_parts):
        body_part = body_part.strip()
        body_args = []
        body_part_arg_cntr = 0
        new_arg = {
            'type': 'input_value',
            'name': f'part_{i}',
            'check': f'kairos_event_{event_name}_part_{i}'
        }
        if i == 0:
            parent_block['args0'].append(new_arg)
        else:
            parent_block[f'message{i}'] = '%1'
            parent_block[f'args{i}'] = [new_arg]
        for j, (arg_name, _) in enumerate(event_body['args'], 1):
            arg_id_str = f'<arg{j}>'
            if arg_id_str in body_part:
                body_part_arg_cntr += 1
                body_part = body_part.replace(arg_id_str, f'%{body_part_arg_cntr}')
                body_args.append({
                    'type': 'input_value',
                    'name': arg_name,
                    'variable': arg_name
                })
        part_block = {
            'type': f'kairos_event_{event_name}_part_{i}',
            'output': f'kairos_event_{event_name}_part_{i}',
            'message0': body_part,
            'args0': body_args,
            'inputsInline': True,
            'style': 'text_blocks',  # TODO: change
            'extensions': ['kairos_events_checkVarsNamesAndTypes']
        }
        part_blocks.append(part_block)
    arg_blocks = []
    for arg_name, _ in event_body['args']:
        arg_blocks.append({
            'type': f'kairos_event_{event_name}_arg_{arg_name}',
            'message0': arg_name,
            'output': 'kairos_var',
            'style': 'variable_blocks',
        })
    
    return [parent_block] + part_blocks + arg_blocks
        

## Generate block JSON descriptions

event_blocks_json = ''

for event_name, event_body in events.items():
    event_blocks = event_to_blocks(event_name, event_body)
    for event_block in event_blocks:
        event_blocks_json += 'Blockly.defineBlocksWithJsonArray([' + json.dumps(event_block) + ']);\n'
    
with open('../kairos_blocks.js', 'w') as fout:
    fout.write(event_blocks_json)

In [17]:
## Generate XML list of blocks

list_of_blocks_xml = '<xml xmlns="https://developers.google.com/blockly/xml" id="toolbox-kairos-blocks" style="display: none">\n'

events_names_dict = collections.defaultdict(list)

for event_name in events.keys():
    event_type = event_name.split('.')[0]
    events_names_dict[event_type].append(event_name)

for event_type, v in events_names_dict.items():
    list_of_blocks_xml += f'<category name="{event_type}" categorystyle="list_category">\n'
    events_names_dict[event_type].sort()
    for event_name in events_names_dict[event_type]:
        list_of_blocks_xml += f'  <block type="kairos_event_{event_name}">\n'
        event_args = events[event_name]['args']
        event_body_parts = events[event_name]['template'].split('|')
        list_of_blocks_xml += f'    <field name=\"step_name\">{to_short_name(event_name)}</field>\n'
        for i, body_part in enumerate(event_body_parts):
            list_of_blocks_xml += f'    <value name="part_{i}"><block type="kairos_event_{event_name}_part_{i}" movable="false" deletable="false">\n'
            for arg_id_str in regex.findall('<arg(\d+)>', body_part):
                arg_id = int(arg_id_str) - 1
                arg_name = event_args[arg_id][0]
                list_of_blocks_xml += f'      <value name="{arg_name}"><shadow type="kairos_event_{event_name}_arg_{arg_name}"></shadow><block type="variables_get"><field name="VAR" variabletype="kairos_var">{arg_name}</field></block></value>\n'
            list_of_blocks_xml += f'    </block></value>\n'
        event_args = events[event_name]['args']
        list_of_blocks_xml += '  </block>\n'
    list_of_blocks_xml += '</category>\n'

# Add control blocks
list_of_blocks_xml += '<sep></sep>\n'
list_of_blocks_xml += '<category name="Control blocks" categorystyle="variable_category">\n'
list_of_blocks_xml += '  <button text="Create variable" callbackKey="CREATE_VARIABLE"></button>'
list_of_blocks_xml += '  <block type="kairos_control_parallel"></block>\n'
list_of_blocks_xml += '  <block type="kairos_control_multivar_2"></block>\n'
list_of_blocks_xml += '  <block type="kairos_control_multivar_3"></block>\n'
list_of_blocks_xml += '  <block type="kairos_control_multivar_4"></block>\n'
list_of_blocks_xml += '  <block type="kairos_control_multivar_5"></block>\n'
list_of_blocks_xml += '</category>\n'
list_of_blocks_xml += '</xml>\n'

with open('../kairos_blocks.xml', 'w') as fout:
    fout.write(list_of_blocks_xml)

---